In [1]:
# Cofiguration
# from MasterThesis.core import config
# config.config_notebook(False)

# Load libraries
from typing import List, Callable, Union, Tuple
import multiprocessing as mp
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import os
import matplotlib.patches as mpatches
import numpy as np
import torch
from tqdm.autonotebook import tqdm

# custom libary
import MasterThesis.preprocessing as DP
from MasterThesis.EDA import image_label_sanity_check
from MasterThesis import EDA
from MasterThesis.regression.metrics import utils

# from MasterThesis.utils import metrics, simclr, data_for_seg
# from MasterThesis.utils import data_for_seg

import wandb
wandb.login(key="ed6b4f7a25cd803c9ce7a66dbfba2353fe5bb5d2")

wandb: Currently logged in as: omar_castno (omar_castano). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/omar/.netrc


True

In [2]:
drive_path = "/media/omar/storage/gdrive"

In [3]:
path_to_label = f"{drive_path}/Maestria/Datasets/GeoDataset/" 
select_classes = ['non_agricultural_area', 'legal_exclusions', 'agricultural_frontier']

In [4]:
#Load metadata
path_to_metadata =  f'{drive_path}/Maestria/Datasets/GeoDataset/metadata/metadata_binary.csv'
metadata = pd.read_csv(path_to_metadata).sample(frac=1, random_state=42)
print(metadata.shape)
metadata.head()

(22581, 5)


,Image,Mask,Patch,non_agricultural(km2),agricultural_frontier(km2)
16110,"18NVN/18NVN_1_1/(300, 800)/chip.npz","LabelsGeoTiff/18NVN/18NVN_1_1_(300, 800).tif",18NVN,22.58,77.42
4140,"18NUP/18NUP_2_6/(300, 700)/chip.npz","LabelsGeoTiff/18NUP/18NUP_2_6_(300, 700).tif",18NUP,78.90,21.10
14107,"18NVM/18NVM_6_7/(0, 300)/chip.npz","LabelsGeoTiff/18NVM/18NVM_6_7_(0, 300).tif",18NVM,58.80,41.20
8182,"18NVK/18NVK_1_1/(700, 800)/chip.npz","LabelsGeoTiff/18NVK/18NVK_1_1_(700, 800).tif",18NVK,27.40,72.60
11403,"18NVL/18NVL_7_3/(400, 100)/chip.npz","LabelsGeoTiff/18NVL/18NVL_7_3_(400, 100).tif",18NVL,95.18,4.82


In [5]:
project = "Regression"
entity = "omar_castano"
name = "report_glcnet"
version = [
    "RandomInit",
    "RandomInit",
    "RandomInit",
    "SimCLR",
    "SimCLR",
    "SimCLR",
]
train_size = [
    0.005,
    0.01,
    0.05,
    0.005,
    0.01,
    0.05,
]


bar_plots = {}
for metric in ["test_RMSE"]:
    plot = utils.barplot_metrics_from_wandb(
        wandb,
        project=project,
        entity=entity,
        version=list(set(version)),
        train_size=list(set(train_size)),
        metric=f"{metric}",
    )

    bar_plots.update({metric: plot})

In [6]:
bar_plots["test_RMSE"]

In [7]:
def standar_report(wandb, project, entity, name, version, train_size):

    bar_plots = {}
    for metric in ["test_RMSE"]:
        plot = utils.barplot_metrics_from_wandb(
            wandb,
            project=project,
            entity=entity,
            version=list(set(version)),
            train_size=list(set(train_size)),
            metric=f"{metric}",
        )

        bar_plots.update({metric: plot})

   
    loss_table = utils.get_table(
        wandb,
        project=project,
        entity=entity,
        version=version,
        train_size=train_size,
        table_name="Loss",
    )

    loss_plot = utils.plot_loss_curves(loss_table, color="train_size", line_dash="version")

    with wandb.init(
        project=project,
        entity=entity,
        config={"version": "report", "train_size": None},
        name=name,
    ) as run:

        [run.log({f"Bareplot {metric}": plot}) for metric, plot in bar_plots.items()]
        run.log({"Loss Plots": loss_plot})

In [8]:
standar_report(
    wandb,
    project="Regression",
    entity="omar_castano",
    name="report",
    version=[
        "RandomInit",
        "RandomInit",
        "RandomInit",
        # "RandomInit",
        "SimCLR",
        "SimCLR",
        "SimCLR",
        # "SimCLR",
    ],
    train_size=[
        0.005,
        0.01,
        0.05,
        # 0.1,
        0.005,
        0.01,
        0.05,
        # 0.1,
    ],
)

-------------------------------------------------------------------------------------
The number of runs found with train_size=0.005 and version=RandomInit is 5
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
The number of runs found with train_size=0.01 and version=RandomInit is 5
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
The number of runs found with train_size=0.05 and version=RandomInit is 5
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
The number of runs found with train_size=0.005 and version=SimCLR is 5
-------------------------------------------------------------------------------------
------------------